## SQL 마지막 실습(sakila)
* 영화 흥행 가장 많이 이룬 (렌탈 매출 가장 높은) 배우 리스트 뽑고, 매출 순으로 정렬하시오
* revenue_per_film_table

### SUB QUERY 사용
* 영화별 그룹 GROUP BY film
* rental table(어떤 영화인지는 inventory_id에있음)
* film table(film_id, title)
* inventory table(inventory_id, film_id)

In [2]:
import MySQLdb
import pandas as pd

db = MySQLdb.connect(
    "db.fastcamp.us",
    "root",
    "dkstncks",
    "sakila",
    charset='utf8',
)

cursor = db.cursor()

###
1. revenue per film table 만들기

** 1. SQL **
* WHERE

In [4]:
SQL_QUERY = """
        SELECT
            film.film_id,
            film.rental_rate * COUNT(rental.rental_id) AS "revenue"
        FROM
            film,
            inventory,
            rental
        WHERE
            rental.inventory_id = inventory.inventory_id
            AND inventory.film_id = film.film_id
        GROUP BY 1
        ;
"""

pd.read_sql(SQL_QUERY, db).tail()

,film_id,revenue
953,996,6.93
954,997,5.94
955,998,8.91
956,999,50.83
957,1000,154.69


** 2. pandas**

In [5]:
inventory_df = pd.read_sql("SELECT * FROM inventory;", db)
rental_df = pd.read_sql("SELECT * FROM rental;", db)
film_df = pd.read_sql("SELECT * FROM film;", db)

In [6]:
# WHERE ... = JOIN => pd.merge

pd.merge(rental_df, inventory_df, on='inventory_id') # WHERE 1

merged_df = pd.merge(
    film_df,
    pd.merge(rental_df, inventory_df, on='inventory_id'), # WHERE 1
    on="film_id",
)
merged_df.tail()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,...,last_update,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update_x,store_id,last_update_y
16039,1000,ZORRO ARK,A Intrepid Panorama of a Mad Scientist And a B...,2006,1,None,3,4.99,50,18.99,...,2006-02-15 05:03:42,6901,2005-07-12 21:46:33,4574,158,2005-07-16 21:36:33,1,2006-02-15 21:30:53,1,2006-02-15 05:09:17
16040,1000,ZORRO ARK,A Intrepid Panorama of a Mad Scientist And a B...,2006,1,None,3,4.99,50,18.99,...,2006-02-15 05:03:42,8419,2005-07-29 06:54:48,4574,37,2005-08-06 05:02:48,1,2006-02-15 21:30:53,1,2006-02-15 05:09:17
16041,1000,ZORRO ARK,A Intrepid Panorama of a Mad Scientist And a B...,2006,1,None,3,4.99,50,18.99,...,2006-02-15 05:03:42,13236,2005-08-19 16:18:24,4574,267,2005-08-27 17:48:24,2,2006-02-15 21:30:53,1,2006-02-15 05:09:17
16042,1000,ZORRO ARK,A Intrepid Panorama of a Mad Scientist And a B...,2006,1,None,3,4.99,50,18.99,...,2006-02-15 05:03:42,10479,2005-08-01 10:11:25,4580,275,2005-08-06 04:52:25,1,2006-02-15 21:30:53,2,2006-02-15 05:09:17
16043,1000,ZORRO ARK,A Intrepid Panorama of a Mad Scientist And a B...,2006,1,None,3,4.99,50,18.99,...,2006-02-15 05:03:42,15916,2005-08-23 17:56:01,4580,327,2005-08-31 21:49:01,2,2006-02-15 21:30:53,2,2006-02-15 05:09:17


In [7]:
import numpy as np

merged_df.groupby('film_id').agg({'rental_rate' : np.sum})

# groupby된거에서 특정col의 로우를 합친것

,rental_rate
film_id,
1,22.77
2,34.93
3,35.88
4,68.77
5,35.88
6,62.79
7,74.85
8,89.82
9,35.88


### 2. 배우별 총 매출
* actor
* film_actor

** 1. SQL**

In [11]:
SQL_QUERY = """
        SELECT
            film.film_id,
            film.rental_rate * COUNT(rental.rental_id) AS "revenue"
        FROM
            film,
            inventory,
            rental
        WHERE
            rental.inventory_id = inventory.inventory_id
            AND inventory.film_id = film.film_id
        GROUP BY 1
        ;
"""

NEW_SQL_QUERY = """
    SELECT
        actor.actor_id,
        CONCAT(
            actor.first_name,
            " ",
            actor.last_name
        ) "Name",
        SUM(revenue_per_film.revenue)
    FROM
        actor,
        film_actor,
        ({SQL_QUERY}) AS revenue_per_film
    WHERE
        film_actor.actor_id = actor.actor_id
        AND revenue_per_film.film_id = film_actor.film_id
    GROUP BY 1
    ORDER BY 3 DESC
""".format(
    SQL_QUERY=SQL_QUERY.replace(";", ""),
)





pd.read_sql(NEW_SQL_QUERY, db)

,actor_id,Name,SUM(revenue_per_film.revenue)
0,107,GINA DEGENERES,2469.47
1,60,HENRY BERRY,1983.88
2,181,MATTHEW CARREY,1935.22
3,81,SCARLETT DAMON,1848.28
4,102,WALTER TORN,1829.60
5,58,CHRISTIAN AKROYD,1800.48
6,144,ANGELA WITHERSPOON,1799.46
7,17,HELEN VOIGHT,1789.43
8,198,MARY KEITEL,1767.26
9,111,CAMERON ZELLWEGER,1760.40


In [12]:
print(NEW_SQL_QUERY)


    SELECT
        actor.actor_id,
        CONCAT(
            actor.first_name,
            " ",
            actor.last_name
        ) "Name",
        SUM(revenue_per_film.revenue)
    FROM
        actor,
        film_actor,
        (
        SELECT
            film.film_id,
            film.rental_rate * COUNT(rental.rental_id) AS "revenue"
        FROM
            film,
            inventory,
            rental
        WHERE
            rental.inventory_id = inventory.inventory_id
            AND inventory.film_id = film.film_id
        GROUP BY 1
        
) AS revenue_per_film
    WHERE
        film_actor.actor_id = actor.actor_id
        AND revenue_per_film.film_id = film_actor.film_id
    GROUP BY 1
    ORDER BY 3 DESC



** 2. pandas**

In [16]:
merged_df.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,...,last_update,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update_x,store_id,last_update_y
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,...,2006-02-15 05:03:42,361,2005-05-27 07:03:28,6,587,2005-05-31 08:01:28,1,2006-02-15 21:30:53,2,2006-02-15 05:09:17
1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,...,2006-02-15 05:03:42,3201,2005-06-21 00:30:26,6,597,2005-06-28 03:42:26,1,2006-02-15 21:30:53,2,2006-02-15 05:09:17


In [48]:
# \(back slash기능 코드속에 엔터가능)
revenue_per_film_df = \
    merged_df.groupby('film_id').agg({'rental_rate' : np.sum})
revenue_per_film_df.head(2)

,rental_rate
film_id,
1,227.70
2,139.72


In [49]:
# film_id 라는 colunm을 만들자

revenue_per_film_df['film_id'] = revenue_per_film_df.index
revenue_per_film_df.head(5)

,rental_rate,film_id
film_id,,
1,227.70,1
2,139.72,2
3,179.40,3
4,343.85,4
5,179.40,5


In [50]:
# FROM ...
actor_df = pd.read_sql("SELECT * FROM actor", db)
film_actor_df = pd.read_sql("SELECT * FROM film_actor", db)

In [51]:
# WHERE...

pd.merge(actor_df, film_actor_df, on="actor_id")

pd.merge(
    pd.merge(actor_df, film_actor_df, on="actor_id"),
    revenue_per_film_df,
    on="film_id"
)

,actor_id,first_name,last_name,last_update_x,film_id,last_update_y,rental_rate
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33,1,2006-02-15 05:05:03,227.70
1,10,CHRISTIAN,GABLE,2006-02-15 04:34:33,1,2006-02-15 05:05:03,227.70
2,20,LUCILLE,TRACY,2006-02-15 04:34:33,1,2006-02-15 05:05:03,227.70
3,30,SANDRA,PECK,2006-02-15 04:34:33,1,2006-02-15 05:05:03,227.70
4,40,JOHNNY,CAGE,2006-02-15 04:34:33,1,2006-02-15 05:05:03,227.70
5,53,MENA,TEMPLE,2006-02-15 04:34:33,1,2006-02-15 05:05:03,227.70
6,108,WARREN,NOLTE,2006-02-15 04:34:33,1,2006-02-15 05:05:03,227.70
7,162,OPRAH,KILMER,2006-02-15 04:34:33,1,2006-02-15 05:05:03,227.70
8,188,ROCK,DUKAKIS,2006-02-15 04:34:33,1,2006-02-15 05:05:03,227.70
9,198,MARY,KEITEL,2006-02-15 04:34:33,1,2006-02-15 05:05:03,227.70


In [53]:
merged_df = pd.merge(
    pd.merge(actor_df, film_actor_df, on="actor_id"),
    revenue_per_film_df,
    on="film_id"
)

merged_df = merged_df.rename(columns={'rental_rate': 'revenue'})

merged_df.groupby('actor_id').agg({'revenue' : np.sum})

,revenue
actor_id,
1,5470.65
2,7685.61
3,5980.94
4,5694.85
5,9636.30
6,6849.18
7,10242.86
8,7796.59
9,4379.50
